# QAOA implementation on Quantum devices

In [17]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install qiskit-terra==0.16.2
!{sys.executable} -m pip install qiskit-aer==0.7.3
!{sys.executable} -m pip install qiskit-ignis==0.5.1
!{sys.executable} -m pip install qiskit-ibmq-provider==0.11.1
!{sys.executable} -m pip install qiskit-aqua==0.8.1
!{sys.executable} -m pip install qiskit==0.23.3

Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


Error processing line 7 of C:\Users\Admin\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\Admin\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


In [13]:
import warnings
help(warnings.filterwarnings)

Help on function filterwarnings in module warnings:

filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)
    Insert an entry into the list of warnings filters (at the front).
    
    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters



In [3]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager
import qiskit

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver,QAOA
from qiskit.aqua.components.optimizers import SPSA, COBYLA,ADAM, L_BFGS_B
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex
from qiskit.optimization.algorithms import MinimumEigenOptimizer,RecursiveMinimumEigenOptimizer

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import math
import networkx as nx
import pandas as pd
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

# importing Qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute
from qiskit.providers.ibmq      import least_busy
from qiskit.tools.monitor       import job_monitor
from qiskit.providers.aer import QasmSimulator

Important prelim functions

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length)).astype(int)
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Feasibility functions

In [4]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n)).astype(int)
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [5]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    
    feasible=False
    while feasible==False and counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        frequency = bestinarray[counter][1]
        counter += 1
        
    if feasible == False:
        return feasible
    else:
        return bestasint[1:], frequency/total

## IBM device loading

In [6]:
from qiskit import IBMQ

# Load account from disk
IBMQ.load_account()
IBMQ.providers()

C:\Users\Admin\Anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='wits-eie')>]

In [7]:
provider = IBMQ.get_provider(hub='ibm-q-wits',project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_manhattan') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_casab

## The QAOA function

In [8]:
def testing_quantum(ins, iterations, backend, optimal_point, trials):
    """
    Input:
    ins: name of instance
    iterations
    backend
    optimal_point
    trials
    
    Output:
    outmatrix : [soln,result time,eigenvector,number of spsa trials]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error

    #QAOA
    machinesolutions = []
    rmachinesolutions = []
    seed = 10598
    mod = QuadraticProgram()
    mod.from_docplex(mdl)
    #print(mod.export_as_lp_string())

    #30 trials
    file = open("thirty_trials-" + str(backend) + "-" + str(ins) ,"w")
    file.write("value,   feasible,   frequency, time, iteration" + "\n")
    ans = np.zeros(5)
    for i in range(trials):
        #try:
        spsa = SPSA(maxiter=iterations)
        #Backend = Aer.get_backend('qasm_simulator')
        backend = provider.get_backend(backend)
        quantum_instance = QuantumInstance(backend=backend,seed_simulator=seed,
                                           seed_transpiler=seed, skip_qobj_validation = False, shots = 1024)

        qaoa = QAOA(qubitOp_docplex,optimizer=spsa, p=3,include_custom=True,initial_point = optimal_point)
        result = qaoa.run(quantum_instance)
        print(result['optimal_point'])
        #print('QAOA vector:', result['eigenstate'])
        print('QAOA time:', result['optimizer_time'])
        print('QAOA QAP objective:', result['eigenvalue'] + offset_docplex)

        n = len(list(result['eigenstate'].values()))
        solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),np.array(list(result['eigenstate'].keys())).reshape(n,1)))
        print(solution)
        for r in solution:
            file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ str(result['optimizer_time']) + "," + str(i) + "\n")
            ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        #except:
            #file.write("An error occurred on iteration " + str(i) + " of the 30 trials")
            #print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the 30 trials")
            #ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
    file.close()
    return ans[1:] 
    

In [9]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

## The results cells

In [ ]:
ins = "made3.csv"
iterations = 100
import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
backend = 'ibmq_manhattan'
optimal_point = read_optimal(ins)
ans = testing_quantum(ins, iterations, backend, optimal_point, 30)

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.16.2',
 'qiskit-aer': '0.7.3',
 'qiskit-ignis': '0.5.1',
 'qiskit-ibmq-provider': '0.11.1',
 'qiskit-aqua': '0.8.1',
 'qiskit': '0.23.3'}